In [138]:
import random
import numpy as np
import math

from deap import base
from deap import creator
from deap import tools


In [139]:
initial_capacities = np.array([5.68666880e-07, 6.88501432e-07, 0.00000000e+00])
low_up_bound = [[0.0, 3.6682175918108224], [0.0, 0.19766913955695925], [0.0, 0.0]]

# initial_capacities = np.array([100, 200, 300])
# low_up_bound = [[0.0, 150], [0.0, 250], [0.0, 350]]



In [140]:
def centroidSP(subpop):
    centroids = []

    # Iterate over each subpopulation and calculate the centroid
    for sub in subpop.values():
        if not isinstance(sub, list) or not all(isinstance(individual, list) for individual in sub):
            raise TypeError("Each subpopulation must be a list of lists (individuals).")
        
        num_solutions = len(sub)  # Number of solutions in the current subpopulation
        num_variables = len(sub[0])  # Number of decision variables
        
        # Calculate the centroid for each decision variable
        centroid = [sum(solution[i] for solution in sub) / num_solutions for i in range(num_variables)]
        centroids.append(centroid)  # Append each centroid to the main list in the required format
    
    return centroids

def fitness(subpop, centroids):
    distances = []
    minimal_distances = []
    fitness_SP = {}

    # Step 1: Calculate Euclidean Distances for each individual
    for q, (subpop_index, subpopulation) in enumerate(subpop.items()):
        subpopulation_distances = []
        
        for individual in subpopulation:
            individual_distances = []
            
            for p, centroid in enumerate(centroids):
                if p != q:  # Skip the centroid of the same subpopulation
                    # Calculate Euclidean distance
                    distance = math.sqrt(sum((individual[i] - centroid[i])**2 for i in range(len(individual))))
                    individual_distances.append(distance)
            
            subpopulation_distances.append(individual_distances)
        
        distances.append(subpopulation_distances)

    # Step 2: Calculate Minimal Distances
    for subpopulation_distances in distances:
        subpopulation_minimal = [min(individual_distances) for individual_distances in subpopulation_distances]
        minimal_distances.append(subpopulation_minimal)

    # Step 3: Calculate Fitness SP for each individual
    for sp_index, subpopulation in enumerate(minimal_distances, start=1):
        fitness_values = [(min_distance,) for min_distance in subpopulation]
        fitness_SP[sp_index] = fitness_values

    return fitness_SP

In [141]:
creator.create("FitnessMaxDist", base.Fitness, weights=(1.0,))  # Fitness to maximize distinctiveness
creator.create("IndividualSP", list, fitness=creator.FitnessMaxDist, cost=0)  # Individual structure in DEAP

def generate_individual():
    adjusted_individual = []
    
    for cap, (low, up) in zip(initial_capacities, low_up_bound):
        if cap == 0:
            # If capacity is zero, set it initially to 1% of the upper bound
            new_value = 0.01 * up
        else:
            # Adjust by a random value between -0.1 and 0.1 of the current value
            adjustment = random.uniform(-0.1, 0.1)
            new_value = cap * (1 + adjustment)
        
        # Ensure the new value is within the lower and upper bounds
        new_value = max(low, min(up, new_value))
        adjusted_individual.append(new_value)
    
    return adjusted_individual

# DEAP toolbox setup
toolbox = base.Toolbox()

# Register the individual and subpopulation initializers
toolbox.register("individualSP", tools.initIterate, creator.IndividualSP, generate_individual)
toolbox.register("subpopulationSP", tools.initRepeat, list, toolbox.individualSP)

c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMaxDist' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Jacob\miniconda3\envs\calliope\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'IndividualSP' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [142]:
# Generate subpopulations with multiple individuals
subpops_unaltered = [toolbox.subpopulationSP(n=2) for _ in range(2)]

subpops_SP = {}

for p in range(2):
    subpops_SP[p+1] = subpops_unaltered[p]

print(subpops_SP)

{1: [[5.238702665677014e-07, 6.984733852418476e-07, 0.0], [6.212133114083466e-07, 7.34855751922312e-07, 0.0]], 2: [[5.235532686017177e-07, 6.296498914693117e-07, 0.0], [5.902718342479442e-07, 7.356252369627608e-07, 0.0]]}


In [134]:
subpops_SP = {1: [[3, 6, 8], [4, 8, 2], [2, 6, 4]], 2: [[4, 9, 2], [3, 6, 9], [1, 4, 0]], 3:[[2, 7, 4], [8, 2, 4], [7, 2, 0]]}

In [143]:
#calculate centroids and fitness
centroids = centroidSP(subpops_SP)
fitness_populations = fitness(subpops_SP, centroids)

In [144]:
print(centroids)
print(fitness_populations)

[[5.72541788988024e-07, 7.166645685820798e-07, 0.0], [5.56912551424831e-07, 6.826375642160363e-07, 0.0]]
{1: [(3.664104005266801e-08,), (8.28331265967036e-08,)], 2: [(9.985704363077665e-08,), (2.595884146800418e-08,)]}
